# Setup Environment

In [2]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import pandas as pd
import numpy as np
import torch
import os
import time

device = torch.device("cuda:0")

# Data

In [3]:
retweets = list(np.array(pd.read_csv(os.getcwd()[:os.getcwd().index("SENTICOIN")] + "SENTICOIN/data/raw/twitter/@elonmusk/csv/retweets.csv")["text"]));
print(retweets[:5]);

['@PoisonJL @elonmusk *cough*', '@sethrosson @elonmusk In Papa Elon we trust.', '@RottenAl @dannybuntu @PoisonJL @elonmusk @dogecoin That’s his fault for buying at the peak, by then It was already too late', '@davidgokhshtein @elonmusk SPREAD THE WORD 🚀 🚀🚀 #dogecoin #dogecointo1dollar #dogecointoadollar #DogecoinToTheMoon #dogeArmy https://t.co/04dGPqTS6T', '@elonmusk Elon the people need one more Doge tweet🙏']


# Model: BerTweet, a RoBERTa Model Trained on English Tweets

In [5]:
tokenizer = AutoTokenizer.from_pretrained("finiteautomata/bertweet-base-sentiment-analysis")
model = AutoModelForSequenceClassification.from_pretrained("finiteautomata/bertweet-base-sentiment-analysis").to(device)

# Predicting Sentiment of @elonmusk's Retweets

In [3]:
dtf_retweets = pd.DataFrame(columns = ["text", "Positive", "Negative", "Neutral"]); dtf_retweets

,text,Positive,Negative,Neutral


In [7]:
for i in range(len(retweets)):
    input = tokenizer(retweets[i], padding=True, truncation=True, return_tensors='pt').to(device)
    predictions = torch.nn.functional.softmax(model(**input).logits, dim=-1);

    if torch.cuda.memory_allocated(0)/torch.cuda.memory_reserved(0) * 100 >= 90:
        print("90% GPU Exceeded!");
    if torch.cuda.memory_allocated(0)/torch.cuda.memory_reserved(0) * 100 >= 95:
        print("95% GPU Exceeded!");

    positive = predictions[:, 0].tolist()
    negative = predictions[:, 1].tolist()
    neutral  = predictions[:, 2].tolist()

    table = {"text":retweets[i],
             "Positive":positive,
             "Negative":negative,
             "Neutral" :neutral}

    temp_static_main = pd.DataFrame(table, columns = ["text", "Positive", "Negative", "Neutral"]);
    dtf_retweets = dtf_retweets.append(temp_static_main);

    print(dtf_retweets.drop("text", axis=1).tail(1));
    del input, predictions, temp_static_main

   Positive  Negative   Neutral
0  0.002705  0.808219  0.189076
   Positive  Negative   Neutral
0  0.000803  0.054679  0.944518
   Positive  Negative   Neutral
0  0.638642  0.350891  0.010467
   Positive  Negative   Neutral
0  0.001683  0.324726  0.673591
   Positive  Negative   Neutral
0   0.00194  0.722701  0.275359
   Positive  Negative   Neutral
0  0.001517  0.334431  0.664053
   Positive  Negative   Neutral
0  0.239671   0.74393  0.016399
   Positive  Negative  Neutral
0  0.001863  0.573698  0.42444
   Positive  Negative   Neutral
0  0.001194  0.067059  0.931747
   Positive  Negative   Neutral
0  0.236804  0.738777  0.024419
   Positive  Negative   Neutral
0   0.00335  0.941443  0.055207
   Positive  Negative   Neutral
0  0.952433  0.044523  0.003044
   Positive  Negative  Neutral
0  0.977314  0.019036  0.00365
   Positive  Negative   Neutral
0  0.909185  0.086684  0.004131
   Positive  Negative   Neutral
0  0.685915  0.307201  0.006884
   Positive  Negative   Neutral
0  0.049837 

In [8]:
dtf_retweets.head(5)

,text,Positive,Negative,Neutral
0,@PoisonJL @elonmusk *cough*,0.002705,0.808219,0.189076
1,@sethrosson @elonmusk In Papa Elon we trust.,0.000803,0.054679,0.944518
2,@RottenAl @dannybuntu @PoisonJL @elonmusk @dog...,0.638642,0.350891,0.010467
3,@davidgokhshtein @elonmusk SPREAD THE WORD 🚀 🚀...,0.001683,0.324726,0.673591
4,@elonmusk Elon the people need one more Doge t...,0.001940,0.722701,0.275359


# Save

In [9]:
dtf_retweets.to_csv(os.getcwd()[:os.getcwd().index("SENTICOIN")] + "SENTICOIN/data/processed/models/sentiment/BerTweet.csv", index=False);